Hi! In this notebook I'm going to describe the data that I'll use to answer to the question: "Which Borough has the less number of "pizzerie"?

First I will collect the data that allows me to identify and locate the Borough of Salerno. This information are collected in the file named "coordinates.csv" that you can see below.

In [3]:
import pandas as pd # library for data analsysis
coordinates=pd.read_csv(r'CAP_SALERNO.csv', sep=";")
coordinates

,CAP,Latitude,Longitude,Unnamed: 3
0,84121,40.677973,14.749906,NaN
1,84122,40.677126,14.765950,NaN
2,84123,40.676365,14.771784,NaN
3,84124,40.681521,14.770326,NaN
4,84125,40.692343,14.749177,NaN
5,84126,40.685770,14.771055,NaN
6,84127,40.671500,14.780536,NaN
7,84128,40.667875,14.789289,NaN
8,84129,40.662482,14.797312,NaN
9,84130,40.682441,14.768096,NaN


Then I'm going to vialize these Borough on the map of Salerno city using the _Folium library_

After the creation of the maps I'm going to dowload the information related to each borought using Foursquare

Here, I'm going to describe the data that I'll for one Borough. The same data, will be dowloaded for all the borougth allowing me to descover how many pizzerie are in each borougth. 

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

print('Libraries imported.')

Libraries imported.


In [5]:
coordinates['Latitude']=coordinates['Latitude'].astype(float)
coordinates['Longitude']=coordinates['Longitude'].astype(float)

In [6]:
CLIENT_ID = 'EKDAJ1PSVFYIVBM0LVHQ04SU2ZNCVDWTU4RUH04EY1OQFIPQ' # your Foursquare ID
CLIENT_SECRET = 'BKFIJAUYK1ZRYGR543ZBBJAT2Z0RTAZOOMU3KGJC3AR3O4XX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EKDAJ1PSVFYIVBM0LVHQ04SU2ZNCVDWTU4RUH04EY1OQFIPQ
CLIENT_SECRET:BKFIJAUYK1ZRYGR543ZBBJAT2Z0RTAZOOMU3KGJC3AR3O4XX


In [7]:
Borough_latitude = coordinates.loc[0, 'Latitude'] # Borough latitude value
Borough_longitude = coordinates.loc[0, 'Longitude'] # Borough longitude value

Borough_name = coordinates.loc[0, 'CAP'] # Borough name

print('Latitude and longitude values of {} are {}, {}.'.format(Borough_name, 
                                                               Borough_latitude, 
                                                               Borough_longitude))

Latitude and longitude values of 84121 are 40.677973, 14.749906.


In [8]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Borough_latitude, 
    Borough_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=EKDAJ1PSVFYIVBM0LVHQ04SU2ZNCVDWTU4RUH04EY1OQFIPQ&client_secret=BKFIJAUYK1ZRYGR543ZBBJAT2Z0RTAZOOMU3KGJC3AR3O4XX&v=20180605&ll=40.677973,14.749906&radius=500&limit=100'

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d11f4bef129b50025e02ddf'},
 'response': {'headerLocation': 'Salerno',
  'headerFullLocation': 'Salerno',
  'headerLocationGranularity': 'city',
  'totalResults': 21,
  'suggestedBounds': {'ne': {'lat': 40.682473004500004,
    'lng': 14.755828588953015},
   'sw': {'lat': 40.6734729955, 'lng': 14.743983411046983}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5134dfaee4b07cce57ca97a7',
       'name': 'Resilienza',
       'location': {'address': 'Via Sabatini, 4',
        'crossStreet': 'Porto di Salerno',
        'lat': 40.67830881974029,
        'lng': 14.750758676877254,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.67830881974029,
          'lng': 14.750758676877254}],
        'distance': 81,
        'postal

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Resilienza,Pizza Place,40.678309,14.750759
1,Teatro Verdi,Theater,40.679121,14.751932
2,Villa Comunale,Garden,40.678409,14.753501
3,Portovecchio,Seafood Restaurant,40.677350,14.751810
4,Spiaggia Santa Teresa,Beach,40.677569,14.755690
5,Bar Verdi,Bar,40.679419,14.752643
6,Galleon Pub,Italian Restaurant,40.679129,14.755055
7,Giardino della Minerva,Historic Site,40.680906,14.753962
8,Bar Umberto,Bar,40.678935,14.755272
9,Hotel VillaPoseidon MaisonDeCharme,Hotel Bar,40.680197,14.749681


In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

21 venues were returned by Foursquare.


As it is show in the table the categories that is of my interest is the "Pizza Place".

To answer my questiom for each Borough I'll take the mean of the frequency of occurrence of each category 